# Data cleaning and merging dataframes

## Loading multiple datasets

### Google way

In [1]:
import pandas as pd

# orderlines.csv
url = 'https://drive.google.com/file/d/14Y7g5ITyf6LMyPoKc9wr010V9StaCUux/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
orderlines = pd.read_csv(path)

# orders.csv
url = 'https://drive.google.com/file/d/1BLEHcP-9fm9Rv7A01H3co2XBMnSr66YC/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
orders = pd.read_csv(path)

# brands.csv
url = 'https://drive.google.com/file/d/1BrNrIY0F1LbyXtyaQygUBXVxQGB3JBqx/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
brands = pd.read_csv(path)

# products.csv
url = 'https://drive.google.com/file/d/1UfsHI80cpQqGfsH2g4T4Tsw8cWayOfzC/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
products = pd.read_csv(path)

df_list = [orderlines, orders, brands, products]
files = ['orderlines','orders','brands','products']

### Classical way

Reading file by file

In [ ]:
# import pandas as pd

# path = '../data/eniac/'
# orderlines = pd.read_csv(path + 'orderlines.csv')
# orders = pd.read_csv(path + 'orders.csv')
# brands = pd.read_csv(path + 'brands.csv')
# products = pd.read_csv(path + 'products.csv')

# df_list = [orderlines, orders, brands, products]
# files = ['orderlines','orders','brands','products']

### Another way

Using `os` and a loop to read all the files from a directory. It's also possible to read only files with a certain extension (like `.csv`):

In [ ]:
# import pandas as pd
# import os
# path = '../data/eniac/'
# path, dirs, files = next(os.walk(path))
# #print(files)

# # remove non-csv files
# for file in files:
#     if not file.endswith("csv"):
#         files.remove(file)

In [ ]:
# # create empty list 
# df_list = []

# # append datasets to the list
# for file in files:
#     temp_df = pd.read_csv(path + file, sep=',')
#     df_list.append(temp_df)

# # show results
# products, orders, orderlines, brands = df_list[0], df_list[1], df_list[2], df_list[3]

## Data quality

### Missing values

In [ ]:
# we can check missing values column
orderlines.isna().sum()

id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64

In [ ]:
# or for the whole dataframe
orderlines.isna().sum().sum()

0

In [ ]:
# the .info() method also tells us the "Non-Null Count" for each column
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


In [ ]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


In [ ]:
brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   short   187 non-null    object
 1   long    187 non-null    object
dtypes: object(2)
memory usage: 3.0+ KB


In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


### Duplicates

The presence of duplicate rows is generally a sign that the data is not clean, and will deserve further exploration.

In [ ]:
orderlines.duplicated().sum()

0

In [ ]:
orders.duplicated().sum()

0

In [ ]:
brands.duplicated().sum()

0

In [ ]:
products.duplicated().sum()

8746

### Data cleanliness - initial assessment

Based on our initial exploration, we know we will need to deal with some missing values. The biggest issue so far are the duplicates on the `products` DataFrame. Here are some aspects that we will need to fix or, at least, explore further:

* **products**: 
    * `price` and `promo_price` are loaded as objects. They should be floats. 
    * Missing values: 
        * `description`: 7 missing values. Maybe that could be inferred from the product name?
        * `price`: the missing values could be filled from the `orderlines` dataset. But first we will need to clean it. 
    * Duplicates: a total of 8746 duplicates seems to indicate this DataFrame has been seriously corrupted.
    
* **orders**: 
    * `created_date` should have a date data type. Then, it would be a good quality check to see if the created date for `orders` mathces with the created dates for `orderlines`. 
    
* **orderlines**: 
    * `unit_price` has to be a float, something is wrong there. 
    * `date` has to be transformed to a date data type. Then, as we said, check that it with matches with the `orders` dataset. 
    
* **brands**: looks fine. 

Where do we have to start? 

1. **Data consistency:** Since `orders` and `orderlines` seem very crucial to the analysis, we will start by cleaning them and checking that the information present in both DataFrames match.

2. **The "products mess":** This file seems to have many issues. We will leave it out for now and perform a proper exploration later to understand better what's going on there.

## Cleaning orders

The data consistency check we will do with `orderlines` will involve two steps: 

* the initial and last dates of the orders should be the same
* the sum of `total_paid` on both datasets should be the same

Let's start by transforming the `created_date` of the orders DataFrame and looking for its earliest and latest values:

In [ ]:
# change date datatype
orders['created_date'] = pd.to_datetime(orders['created_date'])

In [ ]:
# earliest value
min(orders['created_date'])

Timestamp('2017-01-01 00:07:19')

In [ ]:
# latest value
max(orders['created_date'])

Timestamp('2018-03-14 13:58:36')

Now we will look at the overall sum of `total_paid` for the orders table:

In [ ]:
sum(orders['total_paid'])

nan

Why do you think the result of the sum is a nan (not a number)?

In [ ]:
orders.total_paid.isna().sum()

5

There are missing values! We can explore them and see how they are all "Pending" orders:

In [ ]:
orders.loc[orders['total_paid'].isna()]

,order_id,created_date,total_paid,state
127701,427314,2017-11-20 18:54:39,NaN,Pending
132013,431655,2017-11-22 12:15:24,NaN,Pending
147316,447411,2017-11-27 10:32:37,NaN,Pending
148833,448966,2017-11-27 18:54:15,NaN,Pending
149434,449596,2017-11-27 21:52:08,NaN,Pending


Since these orders are only a tiny fraction and there's a valid reason why the `total_paid` value is missing, we will simply exclude them from the dataset:

In [ ]:
orders.dropna(inplace=True)

Now the dataset is clean. And the total paid is: 

In [ ]:
orders['total_paid'].sum()

129159615.07000001

## Cleaning orderlines

Following our data consistency check, will now gather in the orderlines DataFrame the same information we got from orders:

* the initial and last dates
* the sum of `total_paid`


First let's transform our date time:

In [ ]:
orderlines['date'] = pd.to_datetime(orderlines['date'])

In [ ]:
min(orderlines['date'])
# orders: Timestamp('2017-01-01 00:07:19')

Timestamp('2017-01-01 00:07:19')

In [ ]:
max(orderlines['date'])
# orders: Timestamp('2018-03-14 13:58:36')

Timestamp('2018-03-14 13:58:36')

**It's a match!**

Now let's check the `total_paid` for orderlines. It's not going to be as easy as with the orders DataFrame, considering the sturcture of orderlines:


In [ ]:
orderlines.head(3)

,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57



To get this value, we will have to calculate a new column, total price for each row. It would be `product_quantity` * `unit_price`. This operation will require that both columns have a numeric data type:

In [ ]:
orderlines.dtypes

id                           int64
id_order                     int64
product_id                   int64
product_quantity             int64
sku                         object
unit_price                  object
date                datetime64[ns]
dtype: object

In [ ]:
products.head()

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364


...it's not the case right now, so we will have to transform the `unit_price` to a numeric data type. 

In [ ]:
# uncomment the line of code below and read the error it produces:
# orderlines['product_quantity'] * pd.to_numeric(orderlines['unit_price'])

While trying to transform this column to numeric an error appears. From the error message, 

> "Unable to parse string '1.137.99' at position 6"

it seems that our dataset has some problems with the thousands separators: they were encoded as dots, and Python & pandas only admit one dot per number: the _decimal_ separator!

Lesson learned: do not use thousand separators in databases / statistical software / programming languages! Sadly, it's too late for us, and we will have to deal with the issue.

There are many ways to approach this problem. The first thing we will do is to count how many dots appear for each `unit_price` value, using string operations. If there are two or more dots a value, we will consider it corrupted —and either try to fix it, or remove it completely.

In [ ]:
# we create a copy of the dataset
ol_temp = orderlines.copy()

# create a new column with the amount of dots in the unit_price column
ol_temp['dots'] = orderlines['unit_price'].str.count('\.') # the backslash 'escapes' the special meaning of '.' in string operations

# show the rows with more than one dot
ol_temp.query('dots > 1')

,id,id_order,product_id,product_quantity,sku,unit_price,date,dots
6,1119115,299544,0,1,APP1582,1.137.99,2017-01-01 01:17:21,2
11,1119126,299549,0,1,PAC0929,2.565.99,2017-01-01 02:07:42,2
15,1119131,299553,0,1,APP1854,3.278.99,2017-01-01 02:14:47,2
43,1119195,299582,0,1,PAC0961,2.616.99,2017-01-01 08:54:00,2
59,1119214,299596,0,1,PAC1599,2.873.99,2017-01-01 09:53:11,2
...,...,...,...,...,...,...,...,...
293862,1649999,452946,0,1,APP2075,2.999.00,2018-03-14 13:03:33,2
293887,1650045,527321,0,1,PAC2148,3.497.00,2018-03-14 13:10:15,2
293889,1650050,527324,0,1,PAC2117,3.075.00,2018-03-14 13:10:56,2
293911,1650088,527342,0,1,APP2492,1.329.00,2018-03-14 13:24:51,2


Our theory about the thousands separators is confirmed. How can we solve this problem? 

Let's remove all the dots for all the `unit_price`, and then add a dot before the last 2 digits to all the rows. Then we will transform it into numeric values.

##### step 1: remove all dots
a) A "corrupted" price like `1.137.99`	will become `113799`

b) A correct price like `37.99`	will become `3799`

##### step 2: add dots two digits before the end of the number
a) The "corrupted" price will go from `113799` to `1137.99`

b) The correct price will go from `3799` back to `37.99`.

In [ ]:
# step 1: to remove the dots, we replace them for... nothing
orderlines = orderlines.assign(unit_price_nd = orderlines['unit_price'].str.replace('\.','', regex=True))
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374


In [ ]:
# step 2.1: we first separate all numbers between the part that goes before the
# decimal point (integers) and the part that goes afterwards (deimals)
orderlines['integers'] = orderlines['unit_price_nd'].str[:-2]
orderlines['decimals'] = orderlines['unit_price_nd'].str[-2:]
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd,integers,decimals
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899,18,99
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900,399,00
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405,474,05
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839,68,39
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374,23,74


In [ ]:
# we create a copy of the dataset
ol_temp1 = products.copy()

# create a new column with the amount of dots in the unit_price column
ol_temp1['dots'] = products['promo_price'].str.count('\.') # the backslash 'escapes' the special meaning of '.' in string operations

# show the rows with more than one dot
ol_temp1.query('dots == 0')

,sku,name,desc,price,promo_price,in_stock,type,dots
145,PAC0185,"Apple MacBook Pro 133 ""i5 25GHz | RAM 16GB | 2...",Apple MacBook Pro 133 inches (MD101Y / A) with...,1639,1469,0,1282,0
146,PAC0185,"Apple MacBook Pro 133 ""i5 25GHz | RAM 16GB | 2...",Apple MacBook Pro 133 inches (MD101Y / A) with...,1639,1469,0,1282,0
147,PAC0185,"Apple MacBook Pro 133 ""i5 25GHz | RAM 16GB | 2...",Apple MacBook Pro 133 inches (MD101Y / A) with...,1639,1469,0,1282,0
148,PAC0185,"Apple MacBook Pro 133 ""i5 25GHz | RAM 16GB | 2...",Apple MacBook Pro 133 inches (MD101Y / A) with...,1639,1469,0,1282,0
149,PAC0185,"Apple MacBook Pro 133 ""i5 25GHz | RAM 16GB | 2...",Apple MacBook Pro 133 inches (MD101Y / A) with...,1639,1469,0,1282,0
...,...,...,...,...,...,...,...,...
18674,SYN0157-A,Open - Synology RT2600AC Wifi Router AC2600,Refurbished Wifi Wireless Router AC2600 17GHz ...,229.9,222,0,1334,0
18686,IFX0041-A,Open - iFixit P6 Battery Pentalobe screwdriver...,Refurbished devices screwdriver for MacBook Pr...,8.95,6,0,14305406,0
18697,APP0432-A,Open - Apple Lightning connector cable to USB ...,Lightning USB cable 1 meter to charge and sync...,25,18,0,1230,0
18883,PAC2286,"Second hand - Apple LED Cinema Display 24 """,Monitor Refurbished Apple Cinema Display 24 inch,899,499,0,1282,0


In [ ]:
# step 1: to remove the dots, we replace them for... nothing
products = products.assign(promo_price_nd = products['promo_price'].str.replace('\.','', regex=True))
products.head()

,sku,name,desc,price,promo_price,in_stock,type,promo_price_nd
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,499899
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401,589996
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387,569898
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230,229997
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364,3199


In [ ]:
products['integers'] = products['promo_price_nd'].str[:-2]
products['decimals'] = products['promo_price_nd'].str[-2:]
products.head()

,sku,name,desc,price,promo_price,in_stock,type,promo_price_nd,integers,decimals
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,499899,4998,99
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401,589996,5899,96
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387,569898,5698,98
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230,229997,2299,97
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364,3199,31,99


In [ ]:
products['new_promo_price'] = products['integers'] + '.' + products['decimals']
products.head()

,sku,name,desc,price,promo_price,in_stock,type,promo_price_nd,integers,decimals,new_promo_price
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,499899,4998,99,4998.99
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401,589996,5899,96,5899.96
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387,569898,5698,98,5698.98
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230,229997,2299,97,2299.97
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364,3199,31,99,31.99


In [ ]:
# step 2.2: we now concatenate those two parts of the number, with a dot in between
orderlines['new_unit_price'] = orderlines['integers'] + '.' + orderlines['decimals']
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd,integers,decimals,new_unit_price
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899,18,99,18.99
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900,399,00,399.00
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405,474,05,474.05
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839,68,39,68.39
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374,23,74,23.74


We will now try again to convert this column to a numeric data type:

In [ ]:
orderlines['unit_price'] = pd.to_numeric(orderlines['new_unit_price'])
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  float64       
 6   date              293983 non-null  datetime64[ns]
 7   unit_price_nd     293983 non-null  object        
 8   integers          293983 non-null  object        
 9   decimals          293983 non-null  object        
 10  new_unit_price    293983 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(5)
memory usage: 24.7+ MB


Data cleaning done! 

Back to data consistency: Now it is time to multiply `product_quantity` and `unit price`, sum all the rows and check whether the value matches the sum of the `total_paid` from the orders DataFrame: 

In [ ]:
# drop 'auxiliary' columns
orderlines.drop(['unit_price_nd','decimals','integers','new_unit_price'], axis=1, inplace=True)

# create a new column "total_price" by multiplying product_quantity times unit_price
orderlines['total_price'] = orderlines['product_quantity'] * orderlines['unit_price']

# sum of the new column "total_price":
sum(orderlines['total_price'])

128776222.0298095

Sadly, it does not match exactly with the sum of `total_paid` from orders:

In [ ]:
orders['total_paid'].sum()

129159615.07000001

The mismatch is about 383 thousand dollars, a non-neglegible amount of money:

In [ ]:
sum(orderlines['total_price']) - orders['total_paid'].sum()

-383393.040190503

How can we figure out where does the difference come from? 

## Matching `orders` and `orderlines`

It is possible that some orders exist in one dataset but not in the other one. This would be a potential source for this price mismatch. Let's find out!

We first create a new column in the `orderlines` dataset using `assign`. We also use `isin()` to create a boolen value (True/False) that checks whether the `id_order` is present in the `orders` dataset:

In [ ]:
orderlines.assign(check_orders = orderlines['id_order'].isin(orders['order_id']))

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price,check_orders
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,18.99,True
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,399.00,True
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,474.05,True
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,68.39,True
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,23.74,True
...,...,...,...,...,...,...,...,...,...
293978,1650199,527398,0,1,JBL0122,42.99,2018-03-14 13:57:25,42.99,True
293979,1650200,527399,0,1,PAC0653,141.58,2018-03-14 13:57:34,141.58,True
293980,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41,19.98,True
293981,1650202,527388,0,1,BEZ0204,19.99,2018-03-14 13:58:01,19.99,True


Then, using `.query` we select rows where the value in this new column is `False`:

In [ ]:
(
orderlines
    .assign(check_orders = orderlines['id_order'].isin(orders['order_id']))
    .query("check_orders==False")
)

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price,check_orders
5,1119114,295310,0,10,WDT0249,231.79,2017-01-01 01:14:27,2317.90,False
63,1119218,296284,0,1,BNQ0042,699.00,2017-01-01 09:58:35,699.00,False
67,1119223,294806,0,1,APP1849,2558.99,2017-01-01 10:09:15,2558.99,False
69,1119226,294806,0,1,APP1864,2797.99,2017-01-01 10:15:14,2797.99,False
70,1119235,297261,0,1,QNA0177,304.99,2017-01-01 10:17:59,304.99,False
...,...,...,...,...,...,...,...,...,...
275665,1621177,244328,0,1,OWC0260,349.00,2018-02-20 13:00:45,349.00,False
280055,1628699,261391,0,1,APP2352,3343.00,2018-02-27 11:43:25,3343.00,False
280352,1629247,287797,0,1,SYN0182,484.11,2018-02-27 19:16:26,484.11,False
280856,1630150,261391,0,1,SAT0091,44.99,2018-02-28 12:18:26,44.99,False


It looks like 240 rows in `orderlines` come from orders not present in the `orders` dataset. This is quite inconsistent, since the `orders` dataset should be the one and only source of truth for orders: if an order is not there, it should not exist. We will definitely report this anomaly, but for now, let's just remove those "ghost" orders:

In [ ]:
orderlines = (orderlines
              .assign(check_orders = orderlines['id_order'].isin(orders['order_id']))
              .query("check_orders==True"))

Now let's look at this problem in the opposite direction: are there orders in the `orders` dataset not prsent in `orderlines`?

In [ ]:
(orders
 .assign(check_orders = orders['order_id'].isin(orderlines['id_order']))
 .query("check_orders==False"))

,order_id,created_date,total_paid,state,check_orders
8,245941,2017-01-01 10:32:23,183.52,Completed,False
65,268629,2017-01-31 11:27:25,73.23,Completed,False
75,272862,2017-04-18 18:17:10,128.99,Pending,False
82,277655,2017-02-01 08:44:55,9.99,Pending,False
83,277994,2017-01-23 18:30:11,52.99,Completed,False
...,...,...,...,...,...
226835,527328,2018-03-14 13:14:12,0.00,Shopping Basket,False
226851,527344,2018-03-14 13:25:21,0.00,Place Order,False
226853,527346,2018-03-14 13:26:14,0.00,Place Order,False
226855,527348,2018-03-14 13:28:18,0.00,Place Order,False


There are more than 22000 orders in the `orders` dataset that are not present on the `orderlines` dataset!!! We can try to find out why by looking at the state of these orders:

In [ ]:
(orders
 .assign(check_orders = orders['order_id'].isin(orderlines['id_order']))
 .query("check_orders==False")
 ['state'].value_counts())

Place Order        12304
Shopping Basket     9810
Completed             45
Cancelled             41
Pending               13
Name: state, dtype: int64

It looks like most of them are orders that were not fully completed: the products were left in the shopping basket or the order was "placed" but maybe not paid (hence the state "Place Order". Some of them were "Completed", though. 

This will require further research, and we might have to come back to these orders if we have to explore consumer behaviour (e.g. why are orders left in the shopping basket?), but for now, for the sake of data consistency, let's drop all of these unmatched orders:

In [ ]:
orders = (orders
          .assign(check_orders = orders['order_id'].isin(orderlines['id_order']))
          .query("check_orders==True")
         )

Let's now check again if the total paid matches:

In [ ]:
orders['total_paid'].sum()

129130877.18

In [ ]:
orderlines['total_price'].sum()

128659393.77

STILL NOT MATCHING!!! And actually, the difference got larger. This is outrageous. Let's keep exploring.

In [ ]:
orderlines['total_price'].sum() - orders['total_paid'].sum()

-471483.4100000113

## Solving the price mismatch

Let's merge both datasets and compare, order by order, the `total_price`. We will call this new merged dataset `orders_info`.

*Note: Remember that the `orderlines` dataset contains one row per product bought: an order where 3 different products were purchased will result in 3 rows there. Therefore, to merge `orderlines` with `orders`, we have to group `orderlines` by its `id_order` and aggregate it by taking the sum of the `total_price`.

In [ ]:
orders_info = (
orderlines
    .groupby('id_order')
    .agg({'total_price':'sum'})
    .merge(orders, how='left', left_on='id_order', right_on='order_id')
    .copy()
)
orders_info

,total_price,order_id,created_date,total_paid,state,check_orders
0,44.99,241319,2017-01-02 13:35:40,44.99,Cancelled,True
1,129.16,241423,2017-11-06 13:10:02,136.15,Completed,True
2,10.77,242832,2017-12-31 17:40:03,15.76,Completed,True
3,77.99,243330,2017-02-16 10:59:38,84.98,Completed,True
4,153.87,243784,2017-11-24 13:35:19,157.86,Cancelled,True
...,...,...,...,...,...,...
204686,42.99,527397,2018-03-14 13:56:38,42.99,Place Order,True
204687,42.99,527398,2018-03-14 13:57:25,42.99,Shopping Basket,True
204688,141.58,527399,2018-03-14 13:57:34,141.58,Shopping Basket,True
204689,19.98,527400,2018-03-14 13:57:41,19.98,Shopping Basket,True


Now that the `total` from both datasets is in the same dataframe, we can create a new column with the difference:

In [ ]:
orders_info['price_difference'] = orders_info['total_price'] - orders_info['total_paid']
orders_info.sort_values('price_difference').tail(30)

,total_price,order_id,created_date,total_paid,state,check_orders,price_difference
20654,164.90,322147,2017-02-10 11:56:59,125.99,Completed,True,38.91
11725,164.90,312311,2017-01-20 00:59:54,125.99,Completed,True,38.91
45940,44.99,349377,2017-04-24 21:42:28,4.99,Pending,True,40.00
8545,164.90,308825,2017-01-13 14:27:46,123.99,Completed,True,40.91
13784,164.90,314570,2017-01-25 09:01:35,122.99,Completed,True,41.91
21347,164.90,322901,2017-02-12 13:25:12,122.99,Place Order,True,41.91
25517,164.90,327363,2017-02-21 22:42:17,122.99,Completed,True,41.91
986,164.90,300474,2017-01-02 12:24:54,119.00,Completed,True,45.90
166458,214768.02,484334,2018-01-07 15:58:26,214714.31,Shopping Basket,True,53.71
54853,13450.87,359083,2017-05-25 13:58:31,13391.86,Shopping Basket,True,59.01


In [ ]:
orders_info['price_difference'].describe()

count    204691.000000
mean         -2.303391
std         297.461827
min      -90898.610000
25%          -0.010000
50%           0.000000
75%           0.000000
max       36159.900000
Name: price_difference, dtype: float64

Looks like the maximum and minimun price differences are huge: some orders are really corrupted. But we also see from the mean and the quartiles that the price difference is neglegible for most orders.

## Challenge: Remove outliers

Decide on a criteria for removing orders whenever you cannot trust the price difference between `orders` and `orderlines`. 

Note: this solution does not completely achieve 100% trustable data, but the objective here is to end up with the best possible data with a limited amount of time, which makes a complete revision of the database protocols and the data pipelines is not feasable - and business questions are pending. Documenting and reporting this data cleaning process, including the criteria that you will use for determining what do you consider an "outlier", is key. Not being paralized by it is also important!

Using the 1.5 rule

In [ ]:
# identify outliers with interquartile range
from numpy import percentile

# calculate interquartile range
q25, q75 = percentile(orders_info['price_difference'], 25), percentile(orders_info['price_difference'], 75)
iqr = q75 - q25
print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))

# calculate the outlier cutoff
cut_off = iqr * 1.5
lower = q25 - cut_off
upper= q75 + cut_off

# identify outliers
#outliers = [x for x in orders_info['price_difference'] if x < lower or x > upper]
#print('Identified outliers: %d' % len(outliers))

# remove outliers
outliers_removed = [x for x in orders_info['price_difference'] if x >= lower and x <= upper]
print('Non-outlier observations after removal of outliers: %d' % len(outliers_removed))

filtered = orders_info.query('-0.024999999999977263<= price_difference <= 0.014999999999986358')

Percentiles: 25th=-0.010, 75th=0.000, IQR=0.010
Non-outlier observations after removal of outliers: 156882


In [ ]:
filtered.head()

,total_price,order_id,created_date,total_paid,state,check_orders,price_difference
0,44.99,241319,2017-01-02 13:35:40,44.99,Cancelled,True,0.0
5,149.00,245275,2017-06-28 11:35:37,149.00,Completed,True,0.0
7,79.99,245851,2017-04-04 20:58:21,79.99,Pending,True,0.0
11,153.54,247643,2017-05-26 11:50:35,153.54,Completed,True,0.0
13,140.99,251302,2017-05-10 12:27:33,140.99,Completed,True,0.0


In [ ]:
#check upper and lower values seperately and add them to the query
upper

0.014999999999986358

In [ ]:
#Check shape after applying filter to ensure that only rows with values that do not exceed new thresholds are left.
filtered.shape

(156882, 7)

Save the data once you are sure you can trust it!

In [ ]:
#orderlines.to_csv('orderlines_cl.csv', index=False)
#orders.to_csv('orders_cl.csv', index=False)

#from google.colab import files
#files.download("orderlines_cl.csv")
#files.download("orders_cl.csv")

## Challenge: Cleaning products

Now it is time to clean the products dataset. Let's do a quick review of its major problems: 

In [ ]:
print(products.info(), "\n")
print("Missing values:", products.isna().sum(), "\n")
print("Duplicate rows:", products.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   sku              19326 non-null  object
 1   name             19326 non-null  object
 2   desc             19319 non-null  object
 3   price            19280 non-null  object
 4   promo_price      19326 non-null  object
 5   in_stock         19326 non-null  int64 
 6   type             19276 non-null  object
 7   promo_price_nd   19326 non-null  object
 8   integers         19326 non-null  object
 9   decimals         19326 non-null  object
 10  new_promo_price  19326 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.6+ MB
None 

Missing values: sku                 0
name                0
desc                7
price              46
promo_price         0
in_stock            0
type               50
promo_price_nd      0
integers            0
decimals            0
new_promo_pr

Looking at this overview, we can see that there are different things that have to be changed: 

* Data types: 
    * `price` should be a float
    * `promo price` should be a float
* Duplicated rows. They have to be removed. 
    * To accomplish this step you can use the method `pd.DataFrame.drop_duplicates()`. Be sure you drop all the duplicates based on the column **sku**, as it is the one you will use to merge with orderlines. 
* Missing values: 
    * Description maybe can be inferred by the name
    * `price`. Is there a way we can extract the information from another table?
    * `type`. Do we need this column for our analysis?
    
This task can be accomoplished by using all the methods you already know.

### Start of the challenge

#### Duplicates

In [ ]:
products.drop_duplicates("sku",inplace=True)
products.head(3)

,sku,name,desc,price,promo_price,in_stock,type,promo_price_nd,integers,decimals,new_promo_price
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,499899,4998,99,4998.99
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401,589996,5899,96,5899.96
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387,569898,5698,98,5698.98


In [ ]:
products.shape

(10579, 11)

In [ ]:
products.head()

,sku,name,desc,price,promo_price,in_stock,type,promo_price_nd,integers,decimals,new_promo_price
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,499899,4998,99,4998.99
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401,589996,5899,96,5899.96
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387,569898,5698,98,5698.98
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230,229997,2299,97,2299.97
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364,3199,31,99,31.99


Check if you have a unique description for each sku: 

In [ ]:
products.groupby('sku')['desc'].count().sort_values(ascending=False)

sku
8MO0001-A    1
PAC1555      1
PAC1557      1
PAC1558      1
PAC1559      1
            ..
PAC2334      0
HTE0025      0
KAN0034-A    0
APP1622-A    0
WDT0211-A    0
Name: desc, Length: 10579, dtype: int64

The product APP1197 is not unique. Let's zoom in: 

In [ ]:
products.query('sku == "APP1197"')

,sku,name,desc,price,promo_price,in_stock,type,promo_price_nd,integers,decimals,new_promo_price
7992,APP1197,"Apple iMac 21.5 ""Core i5 31 GHz Retina display...",Desktop Apple iMac 21.5 inch i5 31 GHz Retina ...,1729,1305.59,0,1282,130559,1305,59,1305.59


In [ ]:
products.drop_duplicates('sku').query('sku == "APP1197"')

,sku,name,desc,price,promo_price,in_stock,type,promo_price_nd,integers,decimals,new_promo_price
7992,APP1197,"Apple iMac 21.5 ""Core i5 31 GHz Retina display...",Desktop Apple iMac 21.5 inch i5 31 GHz Retina ...,1729,1305.59,0,1282,130559,1305,59,1305.59


I can drop the duplicates only focusing on the column 'sku': 

In [ ]:
products.drop_duplicates('sku', inplace=True)

#### Data types

In [ ]:
 pd.to_numeric(products['price'])

ValueError: ignored

It seems this time that the problem in our products prices is different than the one we have found on orderlines. How is it possible we have more than 3 digits? 

Let's look at a random sample in our dataframe to understant more about it: 

In [ ]:
products.sample(10)

,sku,name,desc,price,promo_price,in_stock,type,promo_price_nd,integers,decimals,new_promo_price
17068,TWS0118,Twelve South BookBook Case Black iPhone 7 Plus,Cover with vintage design book store card slot...,59.99,599.906,0,11865403,599906,5999,06,5999.06
14803,LIF0061-A,Open - Lifeproof nüüd Waterproof Case Black iP...,Waterproof protective cover and extreme condit...,119.99,735.298,0,1298,735298,7352,98,7352.98
17998,STM0101,"Dux STM iPad Case 97 ""Black 2017",Compatible with protective sheath 97 inches Ca...,49.55,369.897,1,12635403,369897,3698,97,3698.97
11334,TRN0011,"MacBook sleeve Sleeve Trunk / Air 13 ""Silver",Neoprene with reinforced zipper MacBook Air 13...,39.95,179.903,0,13835403,179903,1799,03,1799.03
324,OWC0026,"Installation Kit OWC SSD Hard Drive 25 ""Mac mi...",Installation Kit HDD or SSD for Mac mini 2011 ...,54.99,399.905,1,14305406,399905,3999,05,3999.05
15337,REP0334,volume button repair on vibration control and ...,Repair service including parts and labor for i...,6.999.003,69.99,0,"1,44E+11",6999,69,99,69.99
18673,TUC0315-A,"Open - Tucano 105 Pro Cosmo iPad Case ""Blue",Leather reconditioned trunnion up to 360 degre...,39.9,259.844,0,12635403,259844,2598,44,2598.44
2494,OWC0068-A,Open - Mac OWC Memory 8GB 1066MHZ DDR3 SO-DIMM,8GB RAM Mac mini iMac MacBook and MacBook Pro ...,99.99,999.896,0,1298,999896,9998,96,9998.96
19283,AP20468,Like new - Apple iPhone Black Lightning Dock,Support base and refitted with dock connector ...,59,440.004,0,13615399,440004,4400,04,4400.04
15851,BEZ0214,"Be.ez LA robe Case Graphite MacBook Pro 13 ""(L...",flexible sheath with full protection for MacBo...,29.99,199.892,0,13835403,199892,1998,92,1998.92


My strategy would be: 

1. Replace missing values with a "special" string. In that case I will put "000.000" as it would represent that this value has a wrong format and has to be changed. This step is done to avoid future problems while cleaning the column price.
2. Create a new column called "price_split" with the values of price splited by the `.`
3. Create a new column called "dots_count" counting the number of dots a price contains

In [ ]:
import numpy as np

products = (
products
    # Replace all missing values by '000.000'
    .assign(price = lambda x: x['price'].replace(np.nan, '000.000'))
    .assign(price_split = lambda x: x['price'].str.split('\.'),
            dots_count = lambda x: x['price'].str.count('\.'))
)
products.head(10)

,sku,name,desc,price,promo_price,in_stock,type,promo_price_nd,integers,decimals,new_promo_price,price_split,dots_count
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,499899,4998,99,4998.99,"[59, 99]",1
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401,589996,5899,96,5899.96,[59],0
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387,569898,5698,98,5698.98,[59],0
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230,229997,2299,97,2299.97,[25],0
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364,3199,31,99,31.99,"[34, 99]",1
5,APP0073,Apple Composite AV Cable iPhone and iPod white,IPhone and iPod AV Cable Dock to Composite Video.,45,420.003,0,1230,420003,4200,03,4200.03,[45],0
6,KIN0008,Mac Memory Kingston 1GB 667MHz DDR2 SO-DIMM,1GB RAM Mac mini and iMac (2006/07) MacBook Pr...,18.99,146.471,0,1364,146471,1464,71,1464.71,"[18, 99]",1
7,KIN0009,Mac Memory Kingston 2GB 800MHz DDR2 SO-DIMM,2GB RAM iMac with Intel Core 2 Duo (Penryn).,36.99,274.694,0,1364,274694,2746,94,2746.94,"[36, 99]",1
8,KIN0001-2,Mac memory Kingston 4GB (2x2GB) 667MHz DDR2 SO...,RAM 4GB (2x2GB) Mac mini and iMac (2006/07) Ma...,74,669.904,0,1364,669904,6699,04,6699.04,[74],0
9,APP0100,Apple Adapter Mini Display Port to VGA,Adapter Mini Display Port to VGA MacBook and M...,35,330.003,0,1325,330003,3300,03,3300.03,[35],0


Now I need to look at each value in the new column "price_split" and check the folliwing condition: 

If the last element of the value in price split has a length bigger than 2 then add a `True` inside the list `need_check`, else add a `False`. 

Example 1: 

 - Input: price_split = `[[59],[99]]` The last element of the value in price split is `[99]`. The length of this element is 2, as it only have 2 characters. Then we will append a `False` on the list `need_check`. 

Exemple 2: 

- Input: price_split = `[[234],[895]]` The last element of the value in price split is `[895]`. The length of the last element is 3, as it has 3 characters. Then we will append a `True` on the list `need_check`.

In [ ]:
need_check = []
len_split = []
for val in products['price_split']:    
    len_split.append(len(val))
    if len(val[-1]) > 2: 
        need_check.append(True)
    else:
        need_check.append(False)

We have created two lists: 

* `need_check`: tells us which element of the dataframe has a last element on `price_split` with a lenght higher than 2. 
* `len_split` : tells us how many elements are inside each `price_split`. 
    * Example 1: `[[59],[99]]` it has a lenght of 2
    * Example 2: `[59]` it has a lenght of 1
    * Example 3: `[[654],[998]]` it has a lenght of 2
    * Example 4: `[[9],[654],[998]]` it has a lenght of 3
    
Then we can add this elements into our products dataframe to detect these rows: 

In [ ]:
(
products
    .assign(need_check = need_check, 
            len_split = len_split)
    .query('need_check==True & len_split > 1 & dots_count != 0')
).head()

,sku,name,desc,price,promo_price,in_stock,type,promo_price_nd,integers,decimals,new_promo_price,price_split,dots_count,need_check,len_split
34,TWS0019,Twelve South MagicWand support Apple Magic Tra...,MagicWand for wireless keyboard and Magic Trac...,000.000,299.899,0,8696,299899,2998,99,2998.99,"[000, 000]",1,True,2
362,REP0043,Speaker lower repair iPhone 4,Repair service including parts and labor for i...,499.004,499.004,0,"1,44E+11",499004,4990,04,4990.04,"[499, 004]",1,True,2
480,PIE0011,Internal Battery for iPhone 3G,Replacement AC Adapter for Apple iPhone 3G.,98.978,98.978,0,21485407,98978,989,78,989.78,"[98, 978]",1,True,2
515,SEN0061,Sennheiser EZX 80 Handsfree iPhone iPad and iP...,IPhone bluetooth headset with microphone iPad ...,649.891,649.891,0,5384,649891,6498,91,6498.91,"[649, 891]",1,True,2
518,SEV0026,Service installation RAM + HDD + SSD MacBook /...,RAM + HDD installation + SSD in your MacBook /...,599.918,599.918,0,20642062,599918,5999,18,5999.18,"[599, 918]",1,True,2


By defining a set of rules, I can quickly detect the prices that are giving me errors. Those rules are: 

* The length of the last element on the value `price_split` has a lenght bigger than 2 (This is what the column `need_check` is telling us).
* There is more than 1 element inside the column `price_split` (Information given by the column `len_split`). 
* The number of dots in price is different than 0 (Column `dots_count`). 

The next steps would be to use these conditions to replace this values by missing values. To do it, I am going to use the function from the module `numpy` called `np.where`. It works as an `if else` statement; if a condition is accomplished, then return a value, else, return another. 

In [ ]:
products = (
products
    .assign(need_check = need_check,len_split = len_split)
    .assign(
        price = lambda x: pd.to_numeric(np.where((x['need_check']==True) & (x['len_split'] > 1) & (x['dots_count'] != 0), np.nan, x['price'])
        )
    )
)
products.query('need_check==True & len_split > 1 & dots_count != 0').head(10)

,sku,name,desc,price,promo_price,in_stock,type,promo_price_nd,integers,decimals,new_promo_price,price_split,dots_count,need_check,len_split
34,TWS0019,Twelve South MagicWand support Apple Magic Tra...,MagicWand for wireless keyboard and Magic Trac...,NaN,299.899,0,8696,299899,2998,99,2998.99,"[000, 000]",1,True,2
362,REP0043,Speaker lower repair iPhone 4,Repair service including parts and labor for i...,NaN,499.004,0,"1,44E+11",499004,4990,04,4990.04,"[499, 004]",1,True,2
480,PIE0011,Internal Battery for iPhone 3G,Replacement AC Adapter for Apple iPhone 3G.,NaN,98.978,0,21485407,98978,989,78,989.78,"[98, 978]",1,True,2
515,SEN0061,Sennheiser EZX 80 Handsfree iPhone iPad and iP...,IPhone bluetooth headset with microphone iPad ...,NaN,649.891,0,5384,649891,6498,91,6498.91,"[649, 891]",1,True,2
518,SEV0026,Service installation RAM + HDD + SSD MacBook /...,RAM + HDD installation + SSD in your MacBook /...,NaN,599.918,0,20642062,599918,5999,18,5999.18,"[599, 918]",1,True,2
525,SEV0024,Service installation RAM + HDD + SSD Mac mini,installation RAM HDD + SSD + on your Mac mini ...,NaN,599.918,0,20642062,599918,5999,18,5999.18,"[599, 918]",1,True,2
526,SEV0025,Service installation RAM + HDD + SSD iMac,installation RAM HDD + SSD + on your iMac + Da...,NaN,599.918,0,20642062,599918,5999,18,5999.18,"[599, 918]",1,True,2
528,SEV0027,Budget Request repair Apple MacBook Air,diagnosis to repair MacBook Air.,NaN,199.892,0,"1,46E+11",199892,1998,92,1998.92,"[199, 892]",1,True,2
532,PAC0362,"NewerTech pack 2 cups 225 ""for iMac",Two suction pads 225 inches for aluminum iMac.,NaN,169.896,1,14305406,169896,1698,96,1698.96,"[199, 795]",1,True,2
536,SEV0028,Budget Request repair Apple MacBook Pro,diagnosis to repair MacBook Pro.,NaN,199.892,0,"1,46E+11",199892,1998,92,1998.92,"[199, 892]",1,True,2


As you can see, we replaced all the values with wrong prices with missing data. New let's drop all the columns we do not need: 

In [ ]:
products.drop(['price_split','need_check','len_split','dots_count'], axis=1, inplace=True)
products["price"].isna().sum()

And now is time to find a strategy to fill this missing values. 

In [ ]:
products.info()

#### Missing values

As now we have incremented the number of missing data (we transformed all the wrong prices to missing ones) let's find out an strategy to fill them. 

But first, let's look if they are important in our dataset:

In [ ]:
products.assign(row_na = products.isna().sum(axis=1) > 0).query('row_na == False')

In [ ]:
prod_na = products.loc[products.isna().sum(axis=1) > 0,:].copy()
prod_na.head()

Decide which percentage of the data represents. Always think smart about using your time: 

* Which percentage of the orders contains one of these products? In that case I have to look at orderlines.

In [ ]:
orderlines.shape[0]

In [ ]:
# Orders with products with missing prices on products dataset
orderlines.loc[orderlines['sku'].isin(prod_na['sku']),:].shape[0] / orderlines.shape[0]

In [ ]:
# this is interesting, as for the products data it is much smaller
prod_na.shape[0] / products.shape[0]

Which impact has on the total revenue?

In [ ]:
orderlines.loc[orderlines['sku'].isin(prod_na['sku']),:]['total_price'].sum() / orderlines['total_price'].sum()

With a 3% only on the total revenue, but we are considering all the orders, and we are interested on the completed ones. 

In [ ]:
na_skus = prod_na['sku'].tolist()

(
orderlines
    .merge(orders, how='left', left_on='id_order', right_on='order_id')
    .query('state == "Completed" & sku == @na_skus')
    ['total_price'].sum()
) / (
orderlines
    .merge(orders, how='left', left_on='id_order', right_on='order_id')
    .query('state == "Completed"')
    ['total_price'].sum()
)

As you can see, the products that are missing only represents 2.9% of the total invoice. 
I will not drop them, as it still has some information which I would like to keep it to create categories (sku, item name and item description), so I am going to save it with prices as missing values. But I will add an extra column telling me which is the maximum price for these skus in orderlines.

In [ ]:
# find out which sku are missing in products
prod_na_list = (
products
    .loc[products['price'].isna(),'sku']
    .tolist()
)

# find them on orderlines, and aggregate the information to find the maximum value
orderlines_prod_na_max = (
orderlines
    .query("sku == @prod_na_list")
    .groupby('sku') 
    .agg({'unit_price':'max'})
    .rename(columns={'unit_price':'max_price_orderlines'})
    .reset_index()
)
orderlines_prod_na_max[:5]

Time to add the column `max_price_orderlines` in our products data:

In [ ]:
products = (
products
    .merge(orderlines_prod_na_max, how='left',on='sku')
)
products.loc[products['max_price_orderlines'].notna()].sample(10)

And now let's store it as a products clean csv into our path: 

In [ ]:
#products.drop(['promo_price','in_stock'], axis=1).to_csv('products_cl.csv', index=False)
#files.download("products_cl.csv")

## Brands

The brands csv looks fine, so we can work together with it. 

In [ ]:
#brands.to_csv('brands_cl.csv', index=False)
#files.download("brands_cl.csv")